<a href="https://colab.research.google.com/github/TheLazyCactus/ML_Project/blob/main/Final_with_lag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [62]:
df

,Year,Company code,TRIR total,LTIR total
486,2014,ZZ,"3,61",0
456,2014,W,NaN,"0,37"
455,2014,V,"1,41","0,39"
454,2014,U,"1,97","0,49"
453,2014,T,"4,47","0,5"
...,...,...,...,...
31,2023,GG,"0,38","0,18"
30,2023,FF,"0,98","0,18"
29,2023,EE,"0,46","0,18"
27,2023,CC,"0,2","0,2"


In [80]:
df = df.sort_values(by="Year", ascending=True)
#Drop columns
cols_to_remove = ["FAR", "TRIR company only", "TRIR contractor only", "LTIR company only", "LTIR contractor only"]
df.drop(["FAR total", "LTIR company only","LTIR contractor only","TRIR company only", "TRIR contractor only"], axis = 1, inplace=True)

df = df.drop(columns=[col for col in cols_to_remove if col in df.columns])  # List of columns to convert
cols = ["TRIR total", "LTIR total"]
cols_to_process = [col for col in cols if col in df.columns]

# Replace commas with dots and convert to float for the existing columns
df[cols_to_process] = df[cols_to_process].replace(',', '.', regex=True).astype(float)
df[cols] = df[cols].astype(float)

df = df.fillna(0)



#df_total = df[["Year", "Company code", "TRIR total", "LTIR total"]]
#df_company = df[["Year", "Company code", "TRIR company only", "LTIR company only"]]
#df_contractor = df[["Year", "Company code", "TRIR contractor only", "LTIR contractor only"]]

**EDA**

In [69]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

(487, 4)
Index(['Year', 'Company code', 'TRIR total', 'LTIR total'], dtype='object')
Year              int64
Company code     object
TRIR total      float64
LTIR total      float64
dtype: object
Year             10
Company code     59
TRIR total      247
LTIR total      121
dtype: int64
Year            0
Company code    0
TRIR total      0
LTIR total      0
dtype: int64


In [81]:
from sklearn.preprocessing import OneHotEncoder

# Ensure the "Year" column exists and filter only the last 3 years
year_column = "Year"  # Adjust this if your column name is different
if year_column not in df.columns:
    raise KeyError(f"Column '{year_column}' not found. Available columns: {df.columns}")

df_filtered = df[df[year_column].isin([2021, 2022, 2023])]

# Display filtered data
print(df_filtered.head())

# Ensure "Company code" exists before encoding
company_column = "Company code"
if company_column not in df_filtered.columns:
    raise KeyError(f"Column '{company_column}' not found. Available columns: {df_filtered.columns}")

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit-transform the 'Company code' column
encoded_company = encoder.fit_transform(df_filtered[[company_column]])

# Get feature names for encoded categories
company_columns = encoder.get_feature_names_out([company_column])

# Convert to DataFrame
df_encoded = pd.DataFrame(encoded_company, columns=company_columns)

# Merge encoded data back into df_filtered and drop the original "Company code"
df_final = pd.concat([df_filtered.reset_index(drop=True), df_encoded], axis=1)
df_final.drop(columns=[company_column], inplace=True, errors='ignore')


     Year Company code  TRIR total  LTIR total
126  2021   W                 0.81        0.23
117  2021   M                 1.74        0.45
125  2021   V                 0.55        0.27
124  2021   U                 1.46        0.29
123  2021   S                 0.35        0.35


In [82]:
df_final

,Year,TRIR total,LTIR total,Company code_A,Company code_AA,Company code_AAA,Company code_B,Company code_BB,Company code_BBB,Company code_C,...,Company code_V,Company code_VV,Company code_W,Company code_WW,Company code_X,Company code_XX,Company code_Y,Company code_YY,Company code_Z,Company code_ZZ
0,2021,0.81,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021,1.74,0.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021,0.55,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021,1.46,0.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021,0.35,0.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2023,0.38,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,2023,0.98,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,2023,0.46,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152,2023,0.20,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


One hot encoding

In [84]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Create lag features (past 3 years of LTIR & TRIR)
for lag in range(1, 4):
    df_final[f'LTIR_lag_{lag}'] = df_final['LTIR total'].shift(lag)
    df_final[f'TRIR_lag_{lag}'] = df_final['TRIR total'].shift(lag)  # Fixed column name

# Drop rows with NaN values due to shifting
df_final = df_final.dropna().reset_index(drop=True)  # Ensure you are dropping from df_final, not df

# Features (X) and Target (y)
X = df_final.drop(columns=["Year", "LTIR total"])  # Features include company code (one-hot) + past LTIR/TRIR
y = df_final["LTIR total"]  # Target: LTIR

# Train-test split (using last 3 years for testing)
#train_size = len(X) - 3
#X_train, X_test = X[:train_size], X[train_size:]
#y_train, y_test = y[:train_size], y[train_size:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
pred = model.predict(X_test)

print("forest score")
print("MAE", mean_absolute_error(y_test, pred))  # Corrected argument order
#print("RMSE", mean_squared_error(y_test, pred, squared=False))  # Corrected argument order
print("R2 score (Test)", model.score(X_test, y_test))  # Use 'model' instead of 'forest'
print("R2 score (Train)", model.score(X_train, y_train))  # Use 'model' instead of 'forest'



forest score
MAE 0.10235666666666664
R2 score (Test) 0.6523202715256708
R2 score (Train) 0.9633339441029533


***LTIR Prediction***

**With lag**

In [88]:
forest = RandomForestRegressor(n_estimators=200,
                             max_depth=10, random_state=42)
forest.fit(X_train, y_train)
pred = forest.predict(X_test)

print("forest score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score (Test)", forest.score(X_test, y_test))
print("R2 score (Train)", forest.score(X_train, y_train))

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=10),
                               n_estimators=100,
                               max_samples=100,
                                random_state=42)#take 80% of the total samples

bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("Bag score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test, y_test))

gb_reg = GradientBoostingRegressor(max_depth=10,
                                   n_estimators=100,
                                   random_state=42)
gb_reg.fit(X_train, y_train)
pred = gb_reg.predict(X_test)

print("GradBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test, y_test))
print("R2 score", gb_reg.score(X_train, y_train))

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=100,
                            random_state=42)
ada_reg.fit(X_train, y_train)
pred = ada_reg.predict(X_test)
print("AdaBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test, y_test))
print("R2 score", ada_reg.score(X_train, y_train))

forest score
MAE 0.10231150788312982
RMSE 0.03899903671253206
R2 score (Test) 0.656044620823643
R2 score (Train) 0.9592414175218521
Bag score
MAE 0.10058075118835413
RMSE 0.03814329826693931
R2 score 0.6635918800982563
GradBoost score
MAE 0.1117255949496891
RMSE 0.05417201375574864
R2 score 0.5222252367551978
R2 score 0.9999999992653138
AdaBoost score
MAE 0.09462833333333334
RMSE 0.03385764408333334
R2 score 0.7013895780415813
R2 score 0.9992197682066557


Ada boost seems the best model

**Cross validation**

In [94]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_results = cross_val_score(ada_reg, X, y, cv=5, scoring='neg_mean_absolute_error')

# cv_results returns the negative of the score (since lower MAE is better)
# Convert it back to positive values
cv_results = -cv_results

# Print the results
print("ada boost cross validation")
print(f"Cross-validation results for MAE: {cv_results}")
print(f"Average MAE: {np.mean(cv_results)}")
print(f"Standard Deviation of MAE: {np.std(cv_results)}")

ada boost cross validation
Cross-validation results for MAE: [0.09888958 0.08773413 0.061      0.16586207 0.05      ]
Average MAE: 0.09269715585659552
Standard Deviation of MAE: 0.040605022180807616


**Overfitting?**

Compare MAE cross validation to train MAE
If cross validation MAE too high compare to train MAE, model good for training sata but not for new data -> Overfitting

**Hyper param tuning**

In [ ]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_reg = RandomForestRegressor(DecisionTreeRegressor())
model = GridSearchCV(estimator = random_reg, param_grid = grid, cv=5)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
pred = random_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", random_reg.score(X_test, y_test))

Hyper param for adaboost

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error

# Define the parameter grid for GridSearchCV

param_grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}
# Define the AdaBoost model with a DecisionTreeRegressor as base estimator
ada_model = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(),  # Decision tree regressor as base estimator
    random_state=42
)

# Create GridSearchCV, using MAE as the evaluation metric for regression
grid_search = GridSearchCV(ada_model, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Fit the model with the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters and the best cross-validation score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation MAE: {-grid_search.best_score_}")  # Negating the MAE (since it's returned negative)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print test set MAE (Mean Absolute Error) as the evaluation metric
print(f"Test Set MAE: {mean_absolute_error(y_test, y_pred)}")


Best Parameters: {'estimator__max_depth': 10, 'estimator__max_leaf_nodes': 250, 'n_estimators': 100}
Best Cross-Validation MAE: 0.1060138888888889
Test Set MAE: 0.09175238095238095


In [101]:
print("R2 score", best_model.score(X_test, y_test))
print("R2 score", best_model.score(X_train, y_train))

R2 score 0.6992862734595002
R2 score 0.9987426541618256
